# My hangman file

In [2]:
import json
import numpy
import math
import requests
import random
import string
import secrets
import time
import re
import collections
from nltk import ngrams, FreqDist

In [4]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


In [8]:
random.choice(full_dict)

'tertianship'

In [6]:
full_dict_common_letter_sorted = collections.Counter("".join(full_dict)).most_common()

In [7]:
full_dict_common_letter_sorted[0]

('e', 233745)

Ok, the main things I am revising are the n-gram matching and the evaluate guesses. What do they have to do?
-Ok, I match the n-grams in a different way than other people do it, but I think my way is better.
-I need to pad out the left and right for the n-gram for the generated word. The padding is to match for n-grams at the beginning and end I guess
    -maybe I shouldnt do this...
    -However, I have to discount stuff that is like (None a) or (a None) because those will cause issues with the 
-matching only can reccommend ONE letter. the way it is done in the other programs is forcing it into cases... but I can do it a different way.

I am going to break the functions down here, fix them, and paste them back...


In [8]:
full_dictionary = full_dict

#set up n-grams for current dictionary
grams2 = []
grams3 = []
grams4 = []
grams5 = []
for word in full_dictionary:
    #is the word clean? It should be...
    #when I loaded the data it's fine...
    #the API has be a bit nervous so lets just put this here
    word = word.replace(" ", "") #remove any padding that may mess with this

    grams2.extend(list(ngrams(word, 2))) #if I pad left and match NONES with the length 2 n-grams, the program will start guessing first and last letters first...
    grams3.extend(list(ngrams(word, 3, pad_left = True, pad_right = True)))
    grams4.extend(list(ngrams(word, 4, pad_left = True, pad_right = True)))
    grams5.extend(list(ngrams(word, 5, pad_left = True, pad_right = True)))

#Ok, so now need the frequency distributions for these
freq_dist_2 = FreqDist(grams2)
freq_dist_3 = FreqDist(grams3)
freq_dist_4 = FreqDist(grams4)
freq_dist_5 = FreqDist(grams5)

#need to actually extract the data from these distributions to use it
freq_dist_2 = [(item, freq_dist_2.get(item)) for item in freq_dist_2]
freq_dist_3 = [(item, freq_dist_3.get(item)) for item in freq_dist_3]
freq_dist_4 = [(item, freq_dist_4.get(item)) for item in freq_dist_4]
freq_dist_5 = [(item, freq_dist_5.get(item)) for item in freq_dist_5]

words_by_length = {}
min_word_length = 3 #shortest in the full_dict at least
max_word_length = 40 #I guess this is pretty arbitrary

for i in range(min_word_length, max_word_length):
    words_by_length.update({i : []})

#I do it in this kinda roundabout way because maybe there are words of 25 and 28 but no 27 or whatever
#for word in self.full_dictionary:
#    self.full_dictionary[len(word)].append(word)
    

#manually input a word and guess word...
correct_word = "apple"
word = "_pp_e"
guess_list = ["s", "p", "t", "e"]
#failed_letters = ["s", "t"]


#def update_grams(self, word, guess_list):
        #update ngrams to remove items associated with incorrect letters
temp_word = word.replace("_", "")
incorrect_letters = set(guess_list) - set(temp_word)
freq_dist_2 = [(item[0], item[1]) for item in freq_dist_2 if set(item[0]).isdisjoint(set(incorrect_letters))]
freq_dist_3 = [(item[0], item[1]) for item in freq_dist_3 if set(item[0]).isdisjoint(set(incorrect_letters))]
freq_dist_4 = [(item[0], item[1]) for item in freq_dist_4 if set(item[0]).isdisjoint(set(incorrect_letters))]
freq_dist_5 = [(item[0], item[1]) for item in freq_dist_5 if set(item[0]).isdisjoint(set(incorrect_letters))]



    

In [30]:
def find_potential_guesses(freq_N, input_word, guess_list):
        #This takes as input a freq dist for n-grams and a word
        #it outputs all the n-grams from freq dist that match along with the 
        potential_guesses = []
        filtered_dist = []
        if(freq_N == []):
            dimension = 0
        else:
            dimension = len(freq_N[0][0])
        word_ngrams = list(ngrams(input_word, dimension, pad_left = True, pad_right = True))
        #print(dimension)
        #There is an issue though, too much padding, so I will prune the word ngrams to prevent weird overmatching of NONEs
        #print(item[0] for item in word_ngrams)
        #print(word_ngrams)
        word_ngrams = [item for item in word_ngrams if item.count(None) < 2]

        for fe in freq_N:
            #Ok..... so here is the idea...
            #if the ngram in fe matches, then it must be a match 
            #to one of the n-gons of the target word with wildcards
            fe_gram = fe[0]
            occurrences = fe[1]
            
            #now, try to match it to one of the n-grams of the word...
            matched_letters_count = 0 #you must match at least one letter to be considered

            #how about this, I will potentially return guesses that are already in the guess_list
            #and then filter them out in the next program when I go to evaluate the guesses.
            #maybe also worry about the vowel thing there instead of here.

            remove_set = set(guess_list)
            remove_set.add(None)
            
            #print("I am inside the if statement")
            for word_gram in word_ngrams:
                matched_letters_count = 0
                for i in range(dimension):
                    if(word_gram[i] != "_"): #there is a letter here, does the fe match?
                        if(fe_gram[i] == word_gram[i]):
                            matched_letters_count += 1
                        else:
                            #this happens if we have a direct contradiction, immediately reject this
                            matched_letters_count = 0
                            break
                    #we don't have to check what the corresponding letter is if word_gram[i] is blank!
                if(matched_letters_count == dimension - 1): #there should be n-1 matches and 1 guess
                    potential_guesses.append((set(fe_gram) - remove_set, dimension,occurrences))
                    #this is just for testing below vvvvvvvvv
                    #potential_guesses.append((set(fe_gram) - remove_set, fe_gram, dimension,occurrences))

                    #filtered_dist.append(fe)
                    #print(set(fe_gram).issubset(guess_list))
            
        return potential_guesses

In [27]:
find_potential_guesses(freq_dist_2, word, guess_list)

[({'r'}, ('r', 'e'), 2, 24800),
 ({'l'}, ('l', 'e'), 2, 22135),
 ({'n'}, ('n', 'e'), 2, 19584),
 ({'d'}, ('d', 'e'), 2, 13954),
 ({'h'}, ('h', 'e'), 2, 11707),
 ({'m'}, ('m', 'e'), 2, 11077),
 ({'v'}, ('v', 'e'), 2, 10593),
 (set(), ('p', 'e'), 2, 10209),
 (set(), ('p', 'e'), 2, 10209),
 ({'c'}, ('c', 'e'), 2, 9023),
 ({'h'}, ('p', 'h'), 2, 8865),
 ({'r'}, ('p', 'r'), 2, 8590),
 ({'i'}, ('i', 'e'), 2, 8301),
 ({'o'}, ('o', 'p'), 2, 8175),
 ({'g'}, ('g', 'e'), 2, 7326),
 ({'a'}, ('p', 'a'), 2, 7313),
 ({'o'}, ('p', 'o'), 2, 7255),
 ({'i'}, ('p', 'i'), 2, 6063),
 ({'a'}, ('a', 'p'), 2, 6038),
 ({'b'}, ('b', 'e'), 2, 5974),
 (set(), ('e', 'e'), 2, 5426),
 ({'k'}, ('k', 'e'), 2, 5188),
 (set(), ('e', 'p'), 2, 5034),
 ({'l'}, ('p', 'l'), 2, 4323),
 ({'i'}, ('i', 'p'), 2, 4121),
 ({'m'}, ('m', 'p'), 2, 3951),
 ({'z'}, ('z', 'e'), 2, 3531),
 ({'a'}, ('a', 'e'), 2, 3350),
 ({'f'}, ('f', 'e'), 2, 3328),
 ({'w'}, ('w', 'e'), 2, 3228),
 ({'u'}, ('u', 'p'), 2, 2942),
 (set(), ('p', 'p'), 2, 2260),

There are issues with the padding... I need to prune everything that has at least 2 NONEs. That includes the `freq_n` and the ngrams of the word...
Maybe I just need to prune the ones from the word?

In [28]:
find_potential_guesses(freq_dist_4, word, guess_list)

[({'a'}, (None, 'a', 'p', 'p'), 4, 275),
 ({'l'}, ('p', 'l', 'e', None), 4, 135),
 ({'l'}, ('p', 'p', 'l', 'e'), 4, 122),
 ({'r'}, ('p', 'p', 'r', 'e'), 4, 119),
 ({'i'}, ('p', 'p', 'i', 'e'), 4, 66),
 ({'o'}, (None, 'o', 'p', 'p'), 4, 56),
 ({'i'}, ('p', 'i', 'e', None), 4, 42),
 ({'u'}, (None, 'u', 'p', 'p'), 4, 32),
 ({'h'}, ('p', 'h', 'e', None), 4, 26),
 (set(), ('p', 'p', 'e', None), 4, 26),
 (set(), ('p', 'e', 'e', None), 4, 25),
 (set(), (None, None, 'p', 'p'), 4, 10),
 (set(), ('p', 'p', 'e', 'e'), 4, 5),
 ({'n'}, ('p', 'n', 'e', None), 4, 4),
 ({'a'}, ('p', 'a', 'e', None), 4, 4),
 ({'r'}, ('p', 'r', 'e', None), 4, 4),
 (set(), (None, 'e', 'p', 'p'), 4, 4),
 ({'o'}, ('p', 'o', 'e', None), 4, 3),
 ({'u'}, ('p', 'u', 'e', None), 4, 3),
 ({'a'}, ('p', 'p', 'a', 'e'), 4, 2),
 ({'n'}, ('p', 'p', 'n', 'e'), 4, 2),
 ({'c'}, ('p', 'c', 'e', None), 4, 2),
 ({'d'}, (None, 'd', 'p', 'p'), 4, 1),
 ({'l'}, (None, 'l', 'p', 'p'), 4, 1),
 ({'m'}, (None, 'm', 'p', 'p'), 4, 1),
 ({'n'}, (None

In [29]:
find_potential_guesses(freq_dist_5, word, guess_list)

[({'l'}, ('p', 'p', 'l', 'e', None), 5, 37),
 ({'i'}, ('p', 'p', 'i', 'e', None), 5, 13),
 (set(), ('p', 'p', 'e', 'e', None), 5, 2),
 ({'a'}, ('p', 'p', 'a', 'e', None), 5, 1)]

In [31]:
freq_2_guesses = find_potential_guesses(freq_dist_2, word, guess_list)
freq_3_guesses = find_potential_guesses(freq_dist_3, word, guess_list)
freq_4_guesses = find_potential_guesses(freq_dist_4, word, guess_list)
freq_5_guesses = find_potential_guesses(freq_dist_5, word, guess_list)
all_guesses = freq_2_guesses + freq_3_guesses + freq_4_guesses + freq_5_guesses

In [32]:
print(all_guesses[0:10])

[({'r'}, 2, 24800), ({'l'}, 2, 22135), ({'n'}, 2, 19584), ({'d'}, 2, 13954), ({'h'}, 2, 11707), ({'m'}, 2, 11077), ({'v'}, 2, 10593), (set(), 2, 10209), (set(), 2, 10209), ({'c'}, 2, 9023)]


In [36]:
filtered_guesses = [item for item in all_guesses if item[0] != set()] #this filters out all the guesses with no letters in them
sorted_guesses = sorted(filtered_guesses, key = lambda tup: tup[1], reverse = True)
print(sorted_guesses[0:10])

[({'l'}, 5, 37), ({'i'}, 5, 13), ({'a'}, 5, 1), ({'a'}, 4, 275), ({'l'}, 4, 135), ({'l'}, 4, 122), ({'r'}, 4, 119), ({'i'}, 4, 66), ({'o'}, 4, 56), ({'i'}, 4, 42)]


In [50]:
def evaluate_potential_guesses(potential_guesses):
        #each potential guess is in the form:
        #({letters}, dimension from ngram, matched letters count, occurrences of pattern in dict)

        #So, which do we pick:
        #prioritize ngram length first
        #then pick based on most frequent letter
        #I guess how many letters match isnt as important...
        #I have read that vowel ratio may make a difference... maybe implement later

        sorted_guesses = sorted(potential_guesses, key = lambda tup: tup[1], reverse = True)
        if(sorted_guesses != []):
            max_dim = sorted_guesses[0][1]
            #print(max_dim)
            max_dim_guesses = [x for x in sorted_guesses if x[1] == max_dim]
            max_dim_guesses.sort(key = lambda tup: tup[2], reverse = True)

            #print(max_dim_guesses[0])
            max_dim_guesses[0][0]
            max_guess = max_dim_guesses[0]
            #print(max_guess)
            max_guess = list(max_guess)[0] #this is to get the actual string instead of a set
            if len(max_guess) > 0:
                max_guess = list(max_guess)[0]
            
            
            return max_guess #this is a set in general
        else:
            return "!"

In [51]:
evaluate_potential_guesses(all_guesses)

'l'

Ok... now just need a program to sort through all of the different n-grams and pick the best.

In [2]:
from nltk import ngrams, FreqDist

In [1]:
class MyHangman(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.alphabet = list(string.ascii_lowercase)
        self.vowels = list(["a", "e", "i", "o", "u"])
        self.guess_list = []
        self.has_won = False
        self.init_grams()
        #self.init_grams()

    def init_grams(self):
        #set up n-grams for current dictionary
        grams2 = []
        grams3 = []
        grams4 = []
        grams5 = []
        for word in self.full_dictionary:
            #is the word clean? It should be...
            #when I loaded the data it's fine...
            #the API has be a bit nervous so lets just put this here
            word = word.replace(" ", "") #remove any padding that may mess with this

            grams2.extend(list(ngrams(word, 2)))# padding on 2 messes up the guesses.
            grams3.extend(list(ngrams(word, 3, pad_left = True, pad_right = True)))
            grams4.extend(list(ngrams(word, 4, pad_left = True, pad_right = True)))
            grams5.extend(list(ngrams(word, 5, pad_left = True, pad_right = True)))
        
        #Ok, so now need the frequency distributions for these
        freq_dist_2 = FreqDist(grams2)
        freq_dist_3 = FreqDist(grams3)
        freq_dist_4 = FreqDist(grams4)
        freq_dist_5 = FreqDist(grams5)

        #need to actually extract the data from these distributions to use it
        self.freq_dist_2 = [(item, freq_dist_2.get(item)) for item in freq_dist_2]
        self.freq_dist_3 = [(item, freq_dist_3.get(item)) for item in freq_dist_3]
        self.freq_dist_4 = [(item, freq_dist_4.get(item)) for item in freq_dist_4]
        self.freq_dist_5 = [(item, freq_dist_5.get(item)) for item in freq_dist_5]

        self.words_by_length = {}
        min_word_length = 3 #shortest in the full_dict at least
        max_word_length = 40 #I guess this is pretty arbitrary

        for i in range(min_word_length, max_word_length):
            self.words_by_length.update({i : []})

        #I do it in this kinda roundabout way because maybe there are words of 25 and 28 but no 27 or whatever
        #for word in self.full_dictionary:
        #    self.full_dictionary[len(word)].append(word)
        
            
        
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)


    #I think this is an easier way to apply ngrams... I don't really know as much about the subject
        #so there is an equal or greater likelihood that this is completely wrong
    def find_potential_guesses(self, freq_N, input_word, guess_list):
        #This takes as input a freq dist for n-grams and a word
        #it outputs all the n-grams from freq dist that match along with the 
        potential_guesses = []
        filtered_dist = []
        if(freq_N == []):
            dimension = 0
        else:
            dimension = len(freq_N[0][0])
        word_ngrams = list(ngrams(input_word, dimension, pad_left = True, pad_right = True))
        #print(dimension)
        #There is an issue though, too much padding, so I will prune the word ngrams to prevent weird overmatching of NONEs
        #print(item[0] for item in word_ngrams)
        #print(word_ngrams)
        word_ngrams = [item for item in word_ngrams if item.count(None) < 2]
        #print(guess_list)
        remove_set = set(guess_list)
        remove_set.add(None)

        for fe in freq_N:
            #Ok..... so here is the idea...
            #if the ngram in fe matches, then it must be a match 
            #to one of the n-gons of the target word with wildcards
            fe_gram = fe[0]
            occurrences = fe[1]
            
            #now, try to match it to one of the n-grams of the word...
            matched_letters_count = 0 #you must match at least one letter to be considered

            #how about this, I will potentially return guesses that are already in the guess_list
            #and then filter them out in the next program when I go to evaluate the guesses.
            #maybe also worry about the vowel thing there instead of here.
            
            #print("I am inside the if statement")
            for word_gram in word_ngrams:
                matched_letters_count = 0
                for i in range(dimension):
                    if(word_gram[i] != "_"): #there is a letter here, does the fe match?
                        if(fe_gram[i] == word_gram[i]):
                            matched_letters_count += 1
                        else:
                            #this happens if we have a direct contradiction, immediately reject this
                            matched_letters_count = 0
                            break
                    #we don't have to check what the corresponding letter is if word_gram[i] is blank!
                if(matched_letters_count == dimension - 1): #there should be n-1 matches and 1 guess
                    potential_guesses.append((set(fe_gram) - remove_set, dimension,occurrences))
                    #this is just for testing below vvvvvvvvv
                    #potential_guesses.append((set(fe_gram) - remove_set, fe_gram, dimension,occurrences))

                    #filtered_dist.append(fe)
                    #print(set(fe_gram).issubset(guess_list))
            
        return potential_guesses
        
    def evaluate_potential_guesses(self, potential_guesses):
        #each potential guess is in the form:
        #({letters}, dimension from ngram, matched letters count, occurrences of pattern in dict)

        #So, which do we pick:
        #prioritize ngram length first
        #then pick based on most frequent letter
        #I guess how many letters match isnt as important...
        #I have read that vowel ratio may make a difference... maybe implement later

        #ok, need to prune stuff... not pruning enough.

        filtered_guesses = [item for item in potential_guesses if item[0] != set()] #this filters out all the guesses with no letters in them
        sorted_guesses = sorted(filtered_guesses, key = lambda tup: tup[1], reverse = True)

        #sorted_guesses = sorted(potential_guesses, key = lambda tup: tup[1], reverse = True)
        if(sorted_guesses != []):
            max_dim = sorted_guesses[0][1]
            #print(max_dim)
            max_dim_guesses = [x for x in sorted_guesses if x[1] == max_dim]
            max_dim_guesses.sort(key = lambda tup: tup[2], reverse = True)

            #print(max_dim_guesses[0])
            max_dim_guesses[0][0]
            max_guess = max_dim_guesses[0]
            #print(max_guess)
            max_guess = list(max_guess)[0] #this is to get the actual string instead of a set
            if len(max_guess) > 0:
                max_guess = list(max_guess)[0]
            
            
            return max_guess #this is a set in general
        else:
            return "!"
    
    def update_grams(self, freq_dist_list ,word, guess_list):
        #freq_grams is in the form [freq_dist_2, freq_dist_3, freq_dist_4, freq_dist_5]

        #update ngrams to remove items associated with incorrect letters
        word = word.replace("_", "")
        incorrect_letters = set(guess_list) - set(word)
        for freq_dist in freq_dist_list:
            freq_dist = [(item[0], item[1]) for item in freq_dist if set(item[0]).isdisjoint(set(incorrect_letters))]
        #freq_dist_2 = [(item[0], item[1]) for item in freq_dist_2 if set(item[0]).isdisjoint(set(incorrect_letters))]
        #freq_dist_3 = [(item[0], item[1]) for item in freq_dist_3 if set(item[0]).isdisjoint(set(incorrect_letters))]
        #freq_dist_4 = [(item[0], item[1]) for item in freq_dist_4 if set(item[0]).isdisjoint(set(incorrect_letters))]
        #freq_dist_5 = [(item[0], item[1]) for item in freq_dist_5 if set(item[0]).isdisjoint(set(incorrect_letters))]
        return freq_dist_list



    def guess(self, word, freq_dist_list): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # The extra space characters must be coming from the website...
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        #clean_word = word.replace("_",".") #I am using underscores

        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        #print(current_dictionary[0])
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()
        #print(sorted_letter_count)                
        
        guess_letter = '!'

        #print(self.freq_dist_2[0:10])
        #print(self.freq_dist_3[0:10])
        #print(self.freq_dist_4[0:10])
        #print(self.freq_dist_5[0:10])
        #print(self.guessed_letters)
        #print(len(self.guessed_letters))

        #freq_2_guesses = self.find_potential_guesses(self.freq_dist_2, word, self.guessed_letters)
        #freq_3_guesses = self.find_potential_guesses(self.freq_dist_3, word, self.guessed_letters)
        #freq_4_guesses = self.find_potential_guesses(self.freq_dist_4, word, self.guessed_letters)
        #freq_5_guesses = self.find_potential_guesses(self.freq_dist_5, word, self.guessed_letters)
        #print("freq 2 guesses ",freq_2_guesses[0:10])
        #print(freq_3_guesses[0:10])
        #print(freq_4_guesses[0:10])
        #print(freq_5_guesses[0:10])
        #print()

        all_guesses = []
        for fd in freq_dist_list:
            fd_guesses = self.find_potential_guesses(fd, word, self.guessed_letters)
            all_guesses += fd_guesses

        

        #all_guesses = freq_2_guesses + freq_3_guesses + freq_4_guesses + freq_5_guesses
        max_guess = self.evaluate_potential_guesses(all_guesses)
        
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        #print(sorted_letter_count)  
        #print(max_guess)
        #print(sorted_letter_count(max_guess[0])) 
        if(max_guess != "!"):
            #max_guess is in a form like {"a", "b"} or something
            #let's pick the most likely letter from among these
            guess_letter = max_guess
        



        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word

    def get_winstate(self):
        return self.has_won
        
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        #self.init_grams()
        freq_dist_list = [self.freq_dist_2, self.freq_dist_3, self.freq_dist_4, self.freq_dist_5]
        self.has_won = False
        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word, freq_dist_list)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)

            freq_dist_list = self.update_grams(freq_dist_list, word, self.guess_list)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                if verbose:
                    print("Success! the word was: %s" % word)
                self.has_won = True
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                if verbose:
                    print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                if verbose:
                    print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            if verbose:
                print("You Lose, the answer was: %s" % self.correct_word)
                



In [98]:
game = MyHangman()
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: _____.
Guessing letter: e
Incorrect guess e, # of tries remaining: 6. Word: _____.
Guessing letter: i
Incorrect guess i, # of tries remaining: 5. Word: _____.
Guessing letter: a
Got a Letter, a, # of tries remaining: 4. Word: _a_a_.
Guessing letter: l
Incorrect guess l, # of tries remaining: 4. Word: _a_a_.
Guessing letter: n
Incorrect guess n, # of tries remaining: 3. Word: _a_a_.
Guessing letter: c
Got a Letter, c, # of tries remaining: 2. Word: _aca_.
Guessing letter: m
Got a Letter, m, # of tries remaining: 2. Word: maca_.
Guessing letter: r
Incorrect guess r, # of tries remaining: 2. Word: maca_.
Guessing letter: s
Incorrect guess s, # of tries remaining: 1. Word: maca_.
You Lose, the answer was: macap


It looks like it is working but man does it get it wrong a lot...

In [76]:
game.get_winstate()

True

In [9]:
game = MyHangman()

In [17]:
wincount = 0
N = 100
for i in range(N):
    if i % (N/10) == 0:
        print("on game %i of %i" % (i+1, N))
        win_percent = 100*wincount/(i+1)
        print("win percentage: %f" % win_percent)
    game.start_game(verbose=False)
    wincount += game.get_winstate()
print()
print("win percentage: ", 100*wincount/N, "%")

on game 1 of 100
win percentage: 0.000000
on game 11 of 100
win percentage: 63.636364
on game 21 of 100
win percentage: 47.619048
on game 31 of 100
win percentage: 48.387097
on game 41 of 100
win percentage: 53.658537
on game 51 of 100
win percentage: 50.980392
on game 61 of 100
win percentage: 54.098361
on game 71 of 100
win percentage: 50.704225
on game 81 of 100
win percentage: 50.617284
on game 91 of 100
win percentage: 49.450549

win percentage:  49.0 %


In [16]:
print("win percentage: ", 100*wincount/N, "%")

win percentage:  50.0 %


This is SUPER slow... the main thing I think takes forever is just setting up the freq dists... but it should be the same as in the original... so why does this take so much longer?

Maybe I can just save the freq dists, and then when I actually do the program make copies that I prune instead of redoing them EVERY TIME

It takes about 15 seconds per game... which SUCKS REALLY BAD

Well... that is pretty disappointing.